# Saliency Mapper

> A tool to generate area of interests maps of images using image saliency techniques

Code was written by Nicholas M. Synovic, Oscar Yanek, and Rohan Sethi

## Setup

### Upgrade Python `pip` Tool

Upgrade the Python `pip` tool to the latest version

In [ ]:
%pip install --upgrade pip

### Install Python libaries via `pip`

Installed libraries are:

- opencv-contrib-python
- progress

In [ ]:
%pip install opencv-contrib-python progress

### Allow Data to be Loaded From Google Drive

If you wish to load data from Google Drive, uncomment the following lines.

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

## Application

### Import Dependencies 

In [ ]:
from os import listdir
from os.path import join
from pathlib import PurePath

import cv2
from numpy import ndarray
from progress.bar import Bar

### Read Directory

Function to read a directory and return a list of filepaths from that directory.

In [ ]:
def readDirectory(dir: str) -> list:
    files: list = listdir(dir)
    filepaths: list = [join(dir, f) for f in files]
    return filepaths

### Spectral Saliency

Takes a file path to an image (`imagePath`) and an output folder path (default is `./data`; `outputFolder`) as input.

It then uses the approach outlined in [1](#citations) to compute the saliency map of an image.

Area of interest maps are saved in `.jpg` format in the `outputFolder` with the following scheme:

- `outputFolder`/FILENAME_spectralResidual`.jpg`

Where FILENAME is the original name of the file without the extension.


In [ ]:
def computeSpectralSaliency(imagePath: str, outputFolder: str = "data") -> None:
    imageName: str = PurePath(imagePath).with_suffix("").name + "_spectralResidual.jpg"
    outputPath: str = join(outputFolder, imageName)
    
    spectralSaliency = cv2.saliency.StaticSaliencySpectralResidual_create()

    image: ndarray = cv2.imread(imagePath)
    
    (success, saliencyMap) = spectralSaliency.computeSaliency(image)
    
    saliencyMap: ndarray = (saliencyMap * 255).astype("uint8")
    
    cv2.imwrite(outputPath, saliencyMap)

### Fine Grain Saliency

Takes a file path to an image (`imagePath`) and an output folder path (default is `./data`; `outputFolder`) as input.

It then uses the approach outlined in [2](#citations) to compute the saliency map of an image.

Area of interest maps are saved in `.jpg` format in the `outputFolder` with the following scheme:

- `outputFolder`/FILENAME_fineGrain`.jpg`

Where FILENAME is the original name of the file without the extension.

In [ ]:
def computeFineGrainSaliency(imagePath: str, outputFolder: str = "data") -> None:
    imageName: str = PurePath(imagePath).with_suffix("").name + "_fineGrain.jpg"
    outputPath: str = join(outputFolder, imageName)
    
    image: ndarray = cv2.imread(imagePath)
    
    fineGrainSaliency = cv2.saliency.StaticSaliencyFineGrained_create()

    (success, saliencyMap) = fineGrainSaliency.computeSaliency(image)
    saliencyMap: ndarray = (saliencyMap * 255).astype("uint8")
    cv2.imwrite(outputPath, saliencyMap)

### Main Method

In [ ]:
def main() -> None:
    dir: str = input("Image directory to analyze: ")
    imagePaths: list = readDirectory(dir)
    
    with Bar(
        "Creating saliency maps of PascalVOC images...", max=len(imagePaths)
    ) as bar:
        imagePath: str
        for imagePath in imagePaths:
            computeSpectralSaliency(imagePath)
            computeFineGrainSaliency(imagePath)
            bar.next()


if __name__ == "__main__":
    main()


## Citations

1. X. Hou and L. Zhang, “Saliency Detection: A Spectral Residual Approach,” in 2007 IEEE Conference on Computer Vision and Pattern Recognition, Minneapolis, MN, USA, Jun. 2007, pp. 1–8. doi: 10.1109/CVPR.2007.383267.
2. S. Montabone and A. Soto, “Human detection using a mobile platform and novel features derived from a visual saliency mechanism,” Image and Vision Computing, vol. 28, no. 3, pp. 391–402, Mar. 2010, doi: 10.1016/j.imavis.2009.06.006.
